# Running Tximport and DESeq2 to calculate differentially expressed genes between HER2+ and TNBC BC samples

## Review our pipeline
In the pipeline image, the red text indicates the names of the R packages we will use to perform the analysis, the yellow boxes indicate the data (raw or processed), and the grey indicates what we will do in each step. Note that we will use [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) to identify differentially expressed genes (DEGs). This is a highly cited package that uses a negative binomial distribution to calculate DEGs. However, there are multiple pipelines and programs to do this. [Knoweng](http://education.knoweng.org/sequenceng/) has a nice interface to explore other options, including the strengths and weaknesses of each.
![](https://github.com/davidnboone/test-image/blob/master/overview-image.jpg?raw=true "pipeline overview")

## To this point we have:
1. Extracted and calculated HER2 and TNBC status of each patient.
2. Extracted count and abundance data for each transcript from those samples.
3. Prepped the transcipt count and abundance data for Tximport.

## In this notebook we will:
1. Run Tximport, which will collapse count and abundance data to the gene level.
2. Calculate differentially expressed genes using DESeq2.

___
___
___

## We still need the proper R packages loaded. We will need to start by reloading them with the same code as in the previous Notebook.
Of course you can use the typical library() command to load each as well.


In [ ]:
#you can very easily use "install.packages" or "biocLite" to install the packages and "library" to load them
#however, instead I am installing a package called pacman that will determine if a package is already installed
#if it is not it will install it and after will load it
#this method is convenient when sharing code that requires the use of others packages

if (!require("pacman")) install.packages("pacman")
pacman::p_load(R.utils, data.table, tximport, DESeq2, biomaRt, jsonlite, BiocParallel, ggplot2, gplots, RColorBrewer, devtools, pheatmap)


## Set the working directory to the workspace set up in the previous notebook.
 You can check to deterimine if you are in the correct directory by examining your working directory. If it is not the proper workspace then change to proper directory.


In [ ]:
getwd()

In [ ]:
#only need to run if you are not in the proper working directory.
base_dir <- "/Volumes/Oesterreich"
setwd(base_dir)

project_name <- "TCGA_Her2_TNBC_DEGs"
date <- "2019_02_06" #use the same date as notebook 1 and 2
setwd(project_name)


___
___
___

# 1. Collapse transcript-level data to gene level using [Tximport](https://bioconductor.org/packages/release/bioc/html/tximport.html).
If you remember from Notebook 2, DESeq2 calculates differential *gene* expression not *transcript* expression. There are packages that will identify differentially expressed exons or transcripts, however these are still largely unreliable because many isoforms share the same exons, making precise mapping difficult. Accordinly, we need to collapse the transcript counts to gene counts. Tximport is a package that can do this for us. I highly recommend reading the [manual](https://bioconductor.org/packages/release/bioc/html/tximport.html) before starting. 

Tximport requires:
1. counts or tpm data (or output from pseudoaligners Salmon or Kallisto)
2. knowledge file that maps each transcript to gene. 

In notebook 2, we wrangled the count and tpm data into the proper format and created the knowledge file. Now we will run Tximport.

___
___

## 1.1  Read in all of the appropriate  datafiles generated in notebook 1 and 2. 
If you compiled all notebooks into one, and you have the following in your global envirenment, 'brca.clin.selected', 'brca.untran.counts.df', 'brca.untran.tpm.df', 'date', and 'tx2gene81' you can skip the next block of code and go to 1.2 Run Tximport.

As explained above For tximport we need 
1. the unlogged transcript counts data
2. the unlogged transcript tpm data
3. tx2gene81 

After tximport, we will need the clinical data for DESeq2
4. brca.clin.selected

You can read them in using the code below. **Be sure the commands point to the appropriate directories that contain the data**


In [ ]:
#read in the unlogged transcript count data. Because it was saved as an Rda we can use the 
#load() function and it will retain the appropriate variable name
load(file = file.path(date, "df.unlogcounts_her2_tnbc.Rda"))

#read in the unlogged transcript tpm data.
load(file = file.path(date, "df.unlogtpm_her2_tnbc.Rda"))

#read in the tx2gene file
load(file = file.path(date, "tx2gene81.Rda"))

#read in the clinical data
brca.clin.selected <- read.table(file = file.path(date,"brca.clin.data.her2.tnbc.txt"), header = TRUE, stringsAsFactors = FALSE)

___
___

## 1.2 Run Tximport
The resulting file of tximport will provide the **gene** count data. Again, please refer to the Tximport manual to see all the ways to run it. We are providing count and tpm data instead of the output data from a series of data from pseudoaligners, so this is slightly different than if you created your own counts files from fastq files using Salmon or Kallisto. Please, note that in this method, we create a list named txi. This list contains all the abundance data (tpm data that we provide), counts data (again from the counts dataframe generated in Notebook 2), and the length of each transcript. We only need to provide the length data if we are converting counts to tpm or vice versa, which we are not doing. Hence, we will just give dummy values of the same length. We also tell tximport that we are not calculating counts from abundance data for each value. To summarize to gene level, we then provide that list (txi) and the knowledge file tx2gene (generated in Notebook 2).

Before we run tximport, let's confirm that the transcript names in the counts dataframe and tx2gene match.


In [ ]:
#examine transcript names in tx2gene
head(tx2gene81)

#examine transcript names in counts dataframe. Here we are looking at the first 5 rows and 5 columns
brca.untran.counts.df[1:5, 1:5]

**note** the transcript names in the counts dataframe contains the version information. This is the number proceeding the period. tx2gene does not contain this information. We can tell tximport to ignore transcript versions.

In [ ]:
#now to run tximport
#we are only providing the counts and tpm data in the list, but we are inserting in dummy values for length as they are not used in this analysis
#countsFromAbundance flag is set to no because these counts were generated directly from Kallisto and not backwards from tpm data
#abundance is our untransformed transcript tpm data
#counts is our untransformed transcript counts data
txi <- list(abundance = brca.untran.tpm.df, counts = brca.untran.counts.df, length = brca.untran.counts.df, countsFromAbundance = "no")

#txi is the list we made above with our transcript abundance and counts data
#tx2gene81 is the annotation information to translate from transcript to gene
#tx2gene81 was made in Notebook2

df.txi <- summarizeToGene(txi, tx2gene81, ignoreTxVersion = TRUE)

#what structure is df.txi
str(df.txi)



In [ ]:
str(df.txi)

as we can see with the str(df.txi) call, df.txi is a list of all gene-level counts and abundance data for **49922 genes**. Now we need to extract the gene level counts and tpm data from df.txi. We will do this and coerce each into a dataframe for later use.

In [ ]:
#after tximport we need to create variables with the gene counts and tpm
brca.untran.gene.counts <- as.data.frame(df.txi$counts)
brca.untran.gene.tpm <- as.data.frame(df.txi$abundance)


#to view the gene counts data. note that the rownames start with ENSG instead of ENST
message("counts data")
head(brca.untran.gene.counts, nrows = 10)

message("tpm data")
head(brca.untran.gene.tpm, nrows = 10)

#save the data
save(brca.untran.gene.counts, file = file.path(date, "brca.untran.gene.counts.Rda"))
save(brca.untran.gene.tpm, file = file.path(date, "brca.untran.gene.tpm.Rda"))


**note** the tpm data is lower than the counts data for each gene because it is a normalized value correcting for gene length and sequencing depth. [This](https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/) blog does a nice job explaining how the calculation is performed and compares it to two other common RNAseq normalization methods rpkm and fpkm.



## 1.3 Check the results of tximport
Let's also look at the difference between the output of tximport and the input counts file.
First let's identify the transcripts from the first gene listed in the counts file above, "ENSG00000000003".

In [ ]:
gene1_transcripts <- as.vector(tx2gene81[tx2gene81$gene == "ENSG00000000003", 1])
gene1_transcripts

Now let's look at the difference between the output and input of tximport in relation to 'ENSG00000000003'

In [ ]:
message("gene level counts")
brca.untran.gene.counts["ENSG00000000003", ]

message("transcript level counts")
brca.untran.counts.df[gene1_transcripts, ]

You can see that the resulting gene level count data is essentially the sum of all the transcripts. This confirms that tximport worked (at least for this gene).

# 2. Calculate differential expressed genes using [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html)
DESeq2 is a powerful tool that performs differential gene expression analysis based on the negative binomial distribution, using normalized count data. Again, I highly recommend reading the [manual](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) before performing the next steps. There is even a [coursera lecture discussing DESeq2](https://www.coursera.org/lecture/bioconductor/count-based-rna-seq-analysis-XVfPX) that you might want to watch. 




To run DESeq2 we need to specify:
1. A dataframe explaining all samples (TCGA IDs) and their perscribed "group." For ex. TCGA.XX.XXXY.01 is TNBC and TCGA.XX.XXXX.01 is HER2.
2. The counts data with TCGA identifiers that are exactly the same as the dataframe above.
3. The comparison. This is easy for this example because there are only 2 groups and we are calculating the DE genes between those groups.



## 2.1 Before creating the three items needed, let's check that the TCGA identifiers are exactly the same between the counts and clinical data.

In [ ]:
#we need to make sure that TCGA ids are the same in the clinical and counts data
head(brca.clin.selected)
head(brca.untran.gene.counts)



**Note** that the IDs are slightly different. In the clinical data the IDs contain "-" while the counts data IDs contain periods. We need to correct this in our dataframe explaining our samples. We can do that by simply subbing "." for "-" in the clinical data (to match the counts data) using gsub with the identifiers of interest. For us, the IDs of interest are HER2 and TNBC samples that also have counts data. We do this with the blocks of code below.

In [ ]:
#note that the IDs are slightly different. In the clinical data the IDs contain "-" while in the counts data IDs contain "."
#we will need to rectify when running DESeq2

#make vectors of all HER2 positive and TNBC IDs from the clinical data 
#and change - to . to match brca.untran.gene.counts (output of tximport) 
her2.ids <- as.vector(gsub("-", ".", brca.clin.selected[brca.clin.selected$HER2 == "Positive", 1]))
tnbc.ids <- as.vector(gsub("-", ".", brca.clin.selected[brca.clin.selected$TNBC == "TNBC", 1]))

#examine the results of the gsub to confirm it worked

#old clinical data names
message("old clinical data names")
brca.clin.selected[1:5 ,1]

#converted her2 IDs
message("converted  HER2 IDs")
her2.ids[1:5]

#list of IDs from counts data
message("IDs from counts data")
colnames(brca.untran.gene.counts)[1:5]




**Note** the IDs now match, but remember that not all HER2+ and TNBC samples have RNAseq data. If we intersect the column names from the counts data with the converted HER2+ and TNBC IDs, we will identify all identifiers with counts data. 

In [ ]:
#it is possible to have clinical data without rnaseq data so we need to find patient samples that have both
her2.ids.final <- intersect(her2.ids, colnames(brca.untran.gene.counts))
tnbc.ids.final <- intersect(tnbc.ids, colnames(brca.untran.gene.counts))

message("number of HER2 samples")
length(her2.ids)

message("number of HER2 samples with counts data")
length(her2.ids.final)

message("number of TNBC samples")
length(tnbc.ids)

message("number of TNBC samples with counts data")
length(tnbc.ids.final)

message("number of HER2 and TNBC samples with counts")
length(her2.ids.final) + length(tnbc.ids.final)



In [ ]:
#we will need these IDs later, so save them to file
save(her2.ids.final, file = file.path(date, "her2.ids.final.Rda"))
save(tnbc.ids.final, file = file.path(date, "tnbc.ids.final.Rda"))


This is positive reaffirmation that everything is working. As in Notebook 2, we have 206 HER2+ IDs and 176 TNBC IDs. Also, the number of HER2 and TNBC IDs with counts data sums to 338, as it did in Notebook 2. Now we can create what is necessary to run DESeq2:

1. A dataframe explaining all samples (TCGA IDs) and their perscribed "group." For ex. TCGA.XX.XXXY.01	is TNBC and TCGA.XX.XXXX.01 is HER2.
2. The counts data with TCGA identifiers that are exactly the same as the dataframe above. These values also must be integers to run DESeq2.

## 2.2 Create the counts dataframe for DESeq2
The counts dataframe is very straightforward to create now that the IDs match perfectly. We will subset the entire gene.counts data frame to create HER2 positive and TNBC-specific dataframes of the counts. We will then bind them together and floor because DESeq2 expects integers. You could also round the counts, it will not make a noticable difference.

In [ ]:
######################################################################################
############### create counts dataframe for DESeq2 ###################################
######################################################################################

#now create counts dataframes of her2 and tnbc, bind together, and floor because they must be integers for DESeq2
her2.counts.df <- brca.untran.gene.counts[, her2.ids.final] 
tnbc.counts.df <- brca.untran.gene.counts[, tnbc.ids.final]

#bind these together and floor to make all values integers.
her2.tnbc.counts.df <- floor(cbind(her2.counts.df, tnbc.counts.df))

#examine the dataframe
head(her2.tnbc.counts.df)

#check the dimensions to check the correct number of samples and the number of genes
message("dimensions of counts data frame")
dim(her2.tnbc.counts.df)

**note** if the dataframe is correct it should contain 338 samples (as calculated above HER2 positive + TNBC samples with counts data) and 49922 genes (determined by tximport output).

## 2.3 Create knowledge dataframe for DESeq2
The knowledge dataframe is simply a two column dataframe. Column one is the TCGA ID of each sample and column two is their corresponding breast cancer subtype; "HER2" or "TNBC." There are multiple ways to create the dataframe. We will take the simple approach and create a vector that will specify breast cancer subtype by repeating "HER2" by the number of HER2 samples and "TNBC" by the number of TNBC samples. We calculated those numbers in 2.1 above. We will then append that vector to a vector of sample names from the final counts dataframe. **note** it is critical that we join the HER2 and TNBC vectors the same way we joined the counts dataframes above (HER2 then TNBC). We should then check that this is correct. Finally, we also will specify each row name as the same sample name.

In [ ]:
#########################################################################################
################### create knowledge file for DESeq2 see manual ##########################
##########################################################################################

numHER2 <- ncol(her2.counts.df)
numTNBC <- ncol(tnbc.counts.df)

#BC_subtype defines what columns are HER2 samples vs TNBC samples
BC_subtype <- c(rep("HER2", numHER2), rep("TNBC", numTNBC))

#samples of interest are merely the name of the samples that we will examine
samplesofinterest <- colnames(her2.tnbc.counts.df)

#sampleTable is the combination of the above two
#it is a dataframe with sample name and if it is HER2 or TNBC
sampleTable = data.frame(sample = samplesofinterest, BC_subtype = BC_subtype, row.names = samplesofinterest)

#examine the sampleTable
head(sampleTable)

If the dataframe is in the correct format, you should see a dataframe with rownames that are the sample name and two columns -'sample' and'BC_subtype'. Let's confirm the samples are correctly labeled by making sure that the samples labled HER2 and TNBC in the sampleTable correspond to the HER2 and TNBC IDs we calculated above. 

In [ ]:
#subset dataframe to HER2 positive samples and check they are in vector of HER2 IDs
#that we created from the clinical data
message("HER2 logic")
sampleTable[sampleTable$BC_subtype == "HER2", 1] %in% her2.ids.final

#do the same with TNBC IDs
message("TNBC logic")
sampleTable[sampleTable$BC_subtype == "TNBC", 1] %in% tnbc.ids.final



**note** If all samples are correctly labelled, the output from the preceeding code should all be TRUE 

## 2.4 Run DESeq2
There are two steps to run DESeq2. The first (2.4a) is to create the object that DESeq2 uses for the statistical tests. The object class used by the DESeq2 package to store the read counts and the intermediate estimated quantities during statistical analysis is the DESeqDataSet. One caveat is that the counts data must be in a matrix instead of a dataframe. The second step (2.4b) is performing the statistical analysis, which is the most time consuming step.

### 2.4a Creating the DESeqDataSet

In [ ]:
#########################################################################################
################### Run the first step of DESeq2 ##########################
##########################################################################################


#her2.tnbc.counts.df is our final count matrix from her2 and tnbc samples only
#sample table defines the subtpe for each sample
#design in this case is simply HER2 vs TNBC, which is definied in BC_subtype of the sampleTable. This can be much more complicated depending on your samples
#simply put, the design sets up the reference group for the differential expression analysis
deseqdata <- DESeqDataSetFromMatrix(countData = as.matrix(her2.tnbc.counts.df), colData = sampleTable, design = ~ BC_subtype)



**note** Taken directly from the reference [manual](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#the-deseqdataset), "By default, R will choose a reference level for factors based on alphabetical order. Meaning, if you do not specify which level you want to compare against (e.g. which level represents the control group), the comparisons will be based on the alphabetical order of the levels. We can manually set the comparison group using the relevel function on the deseqdata. You should only change the factor levels of variables in the design before running the DESeq2 analysis, not during or afterward." 

Accordingly, we will set the HER2 group as the reference label explicitly with the code below to avoid confusion when interpreting the results.

In [ ]:
#the relevel function sets HER2 as the reference level for fold change calculations
deseqdata$BC_subtype <- relevel(deseqdata$BC_subtype, ref = "HER2")

#now save deseqdata in case we ever need to start from here again
save(deseqdata, file = file.path(date,"her2vstnbc.deseq.Rda"))

### 2.4b Perform the differential expression analysis with DESeq2
This is the most time and computational consuming portion of this analysis. Before we begin, we will clear the global environment and RAM except for the necessary items.

In [ ]:
# to view what is in the global environment before clearing
message("in global environment before rm")
ls()

#remove large elements that are not needed. note we need tissuesource, samplesofinterest, and brca.untran.gene.tpm.df
rm(brca.untran.counts.df, brca.untran.gene.counts, her2.counts.df, brca.untran.tpm.df, df.txi, tx2gene81, tnbc.counts.df, txi)
gc()

#to make sure it worked
message("in global environment after rm")
ls()

#to free up ram
gc()



### Now we are ready to perform the statistical analysis. For the purposes of this exercise we will remove genes that are expressed at a low level across many samples to drastically increase the speed of the analysis. However at the end of this notebook, I include the code  (all commented out) that I normally would run that allows DESeq2 to perform a statistical test across all genes instead of arbitrary filtering. 
Note that we use the SnowParam function to make the code run across multiple threads. The number of threads depends on your machine and if you want to dedicate it completely to this task. On a relatively new laptop this step should not take more than a few hours. However, you might want to run overnight.

The verbose output should read something like:
```
using pre-existing size factors
estimating dispersions
gene-wise dispersion estimates: 4 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 4 workers
-- replacing outliers and refitting for 502 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing
```


In [ ]:
##################################################################################################
################ code to execute DEseq2 after removing lowly expressed genes######################
################ this is only done to decrease computational time.  ###############################
############### normally run the code above and would skip this     ###############################
###################################################################################################

#will make DESeq2 run across 4 threads. You can change the number if running on a machine/server with more or less than 4.
numCores <- 4
register(SnowParam(numCores))


#these commands remove genes with low counts
#the first number specifies the number of counts for a given gene necessary to include in the deseq analysis
#the second number tells how many samples need to have that mean counts
#so in this case only genes with at least 200 counts in 100 samples will be kept
#this is much higher than I would normally do
#these numbers were selected only to reduce computational time for this exercise
#Normally, filtering is not necessary before running
dds <- estimateSizeFactors(deseqdata)
idx <- rowSums(counts(dds, normalized=TRUE) >= 200 ) >= 100
dds <- dds[idx,]

#this is the DESeq execution command that will take ~1hr
dds <- DESeq(dds, parallel = TRUE)

#save after running to save computational time later
save(dds, file = file.path(date, "her2.tnbc.results.Rda"))



### Create an output directory to save all tables and graphs and make the differential expression results readable to any user

In [ ]:
#make an output director
outdir <- "DESeq_output"
dir.create(file.path(date, outdir), recursive = TRUE)


#the results function summarizes the deseq results
#anythign with an alpha -FDR- greater than 0.001 is not considered significant
#you can obviously change this threshold so that everything is included or to be
#more stringent
results.fdr.threshold <- results(dds, alpha = 0.001, parallel = TRUE)
save(results.fdr.threshold, file = file.path(date, outdir, "her2.tnbc.results.Rda"))

#make it a dataframe so that we can export it to a csv
results.fdr.threshold.df <- as.data.frame(results.fdr.threshold)

#reorder based on pvalue
results.ordered <- results.fdr.threshold.df[order(results.fdr.threshold.df$pvalue, na.last = TRUE),]

write.csv(results.ordered, file = file.path(date, outdir, "her2.tnbc.results.csv"))

#check what results file looks like
head(results.ordered)

### Print to screen simple summary statistics
In R, I save the summary below to file by using the sink command that is not compatible in Jupyter Notebooks



In [ ]:
#print summary of deseq to screen. 
#in native R, save this output by uncommenting the sink commands


#sink(file.path(date, "DESeqsummary_HER2_TNBC.txt"))
#note this is a summary  of the results BEFORE making it a dataframe
message("summary of dds")
summary(results.fdr.threshold)
#sink()

#this is a summary of the dataframe
#notice the difference in the summary statistics
message("summary of data table")
summary(results.ordered)

**note** this provides  a lot of information. First, there are only 14,807 genes with non-zero counts. This means we filtered a lot out before the differential expressional analysis. This number will be different if you run the code at the end of this notebook that do not exclude genes expressed at low levels. This summary also tells us that there are 3996 genes significantly upregulated in TNBC and 3375 significantly downregulated (FDR <0.001). 

The summary stats of the data table also tell us a great deal. The log2FCs range from -6.8 to 5.87. The padj column tells us that at least a quarter of the genes have an FDR of <0.000001 meaning the HER2 and TNBC groups are very different.

# The differential expression statistical analysis is complete.
In the next notebook, we will annotate the results file and visually explore the data.


___
___
___


## Below is the code to execute the differential expression analysis without ignoring genes expressed at low levels. Without a time restriction, I recommend doing this because arbitrary cutoffs will lead to false negatives.

In [ ]:
########################################################################################
################### execute DESeq2 #####################################################
#########################################################################################

#Normally I would run everything below, but this will take ~4 hours on 4 threads on this server
#accordingly, run the next code set that will remove genes with low expression, which will decrease
#the computational time to ~1hr.


load(file = file.path(date,"her2vstnbc.deseq.Rda"))

#will run across 4 threads
numCores <- 4
register(SnowParam(numCores))

dds.total <- DESeq(deseqdata, parallel = TRUE)
save(dds.total, file = "her2.dds.total.Rda")
Sys.time()

fdr = .001
results.total <- results(dds.total, alpha = fdr, parallel = TRUE)
save(results.total, file = file.path(date, "her2.tnbc.total.results.Rda"))

#reorder based on pvalue
results.total.df <- as.data.frame(results.total)
results.total.ordered <- results.total.df[order(results.total.df$pvalue, na.last = TRUE),]

write.csv(results.total.ordered, file = file.path(date, outdir, "her2.tnbc.results.total.ordered.csv"))

